In [29]:
import sys

sys.path.insert(0,'..')

from utils.comet_class import Comet
import pandas as pd
import ast
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


print("model loading ...")
comet = Comet('../../lib/comet-atomic-2020/models/comet_atomic2020_bart/comet-atomic_2020_BART')
comet.model.zero_grad()
print("model loaded")


model loading ...
model loaded


In [30]:
queries = []
head = "PersonX went to the Hospital"
rel = "xWant"
query = "{} {} ".format(head, rel)
queries.append(query)
print(queries)
results = comet.generate(queries, decode_method="top_k", num_generate=10)
print(results)

['PersonX went to the Hospital xWant ']
[[' to go home', ' to be treated at the hospital', ' to get a bandaid on their wound.', ' to be treated for their ailment', ' to be taken to the hospital by a doctor.', ' to get better', ' to be taken to the hospital by a doctor.', ' to get a good report from the doctor.', ' to be taken to the hospital by a doctor.', ' to get checked out by the doctor']]


In [31]:
roc_stories_triples_csv = '../../generated/knowledge-triple-extraction/ROCStories_resolved_with_knowledge_triples.csv'
roc_stories_triples_df = pd.read_csv(roc_stories_triples_csv, sep='\t', header=0)

In [32]:
roc_stories_triples_df = roc_stories_triples_df.iloc[:1]

In [33]:
del roc_stories_triples_df['Unnamed: 0']
del roc_stories_triples_df['Unnamed: 0.1']
del roc_stories_triples_df['Unnamed: 0.1.1']
del roc_stories_triples_df['Unnamed: 0.1.1.1']

In [34]:
del roc_stories_triples_df['srl_r1']

In [8]:
from itertools import chain
import multiprocessing as mp
from tqdm import tqdm

In [9]:
forward_rels = ["xWant", "isBefore", "xEffect"]
backward_rels = ["xIntent", "isAfter", "xNeed"]

In [10]:
%%time
result_df = pd.DataFrame()
story_ids = []
sentence_ids = []
results_final = []
for n in range(1,6):
    print(f"n = {n}")
    tmp_df = pd.DataFrame()
    new_events_rows = []
    queries = []
    relations = []
    triples_res = []

    for index, row in tqdm(roc_stories_triples_df.iterrows()):
        story_id = row['storyid']
        sentence_id = f'sentence_{n}'
        triples = ast.literal_eval(row[f'triples{n}']) 
        for idx, triple in enumerate(triples):
            comet_head = ' '.join(triple)
#             print(comet_head)
            if n < 5:
                for rel in forward_rels:
                    query = "{} {} ".format(comet_head, rel)
                    queries.append(query)
                    relations.append(rel)
                    triples_res.append(triple)
                    # for pivot
                    story_ids.append(story_id)
                    sentence_ids.append(sentence_id)
            if n > 1:
                for rel in backward_rels:
                    query = "{} {} ".format(comet_head, rel)
                    queries.append(query)
                    relations.append(rel)
                    triples_res.append(triple)
                    # for pivot
                    story_ids.append(story_id)
                    sentence_ids.append(sentence_id)
    
    print('Genrating events...')
    results = comet.generate(queries, decode_method="beam", num_generate=10)
    results = [list(filter(lambda x: x.strip() != 'none', r)) for r in results]
    new_events = [list(a) for a in zip(triples_res,relations, results)]
    
    print(len(new_events))
    results_final += new_events

    print(len(results_final))
    
result_df['storyid'] = story_ids
result_df['sentence_id'] = sentence_ids
result_df[f'new_events'] = results_final
    
#     result_df.to_csv()

n = 1


1it [00:00, 2110.87it/s]

Genrating events...


3
3
n = 2


1it [00:00, 2407.75it/s]

...

Genrating events...


12
45
n = 5


1it [00:00, 914.19it/s]

Genrating events...


6
51
CPU times: user 40.9 s, sys: 5.38 s, total: 46.2 s
Wall time: 36.6 s


In [11]:
# result_df

In [12]:
tmp = result_df.groupby(['storyid', 'sentence_id']).new_events.apply(list).to_frame().reset_index()

In [13]:
res = tmp.pivot(index='storyid', columns='sentence_id', values='new_events').reset_index()

In [14]:
res.columns.name = None

In [35]:
res[res['storyid']=='000d05af-5db1-4d9d-b4e2-6c01b108b0b1']

,storyid,sentence_1,sentence_2,sentence_3,sentence_4,sentence_5,pruned_comet1,pruned_comet2,pruned_comet3,pruned_comet4


In [16]:
def forward_filter(triple):
    return triple[1] in forward_rels

In [17]:
def backward_filter(triple):
    return triple[1] in backward_rels

In [18]:
def get_forward_triples(triples, index):
    forward_events = []
    for triple in triples:
        if triple[1] == forward_rels[index]:
            forward_events.append(triple)
    return forward_events

In [19]:
def get_backward_events(triples, index):
    backward_events = []
    for triple in triples:
        if triple[1] == backward_rels[index]:
            backward_events += triple[2]
    return [a.strip() for a in backward_events]

In [25]:
pruned_triples = [[] for _ in range(4)]

In [28]:
n_rels = len(forward_rels)
for idx, row in tqdm(res.iterrows()):
    print(f'row {idx}')
    for f in range(1,5):
        print(f'row {idx}, sentence {f}')
        triples_f = row[f'sentence_{f}']
        forward_events = list(filter(forward_filter, triples_f))
        backward_events = []
        for b in range(f+1,6):
            triples_b = row[f'sentence_{b}']
            backward_events += list(filter(backward_filter, triples_b))
        pruned_triples_f = []
#         print(backward_events)
        for rel in range(n_rels):
            print(f'row {idx}, sentence {f}, relation {forward_rels[rel]}')
            ft = get_forward_triples(forward_events, rel)
#             print(ft)
            be = get_backward_events(backward_events, rel)
#             print(be)
            new_ft = []
            for triple in ft:
#                 print(triple)
#                 print(triple)
#                 print(type(triple[2]))
                triple_copy = [a for a in triple]
                triple_copy[2] = [event.strip() for event in triple[2] if event.strip() in be]
                print(triple_copy)
                if triple_copy[2]:
                    new_ft.append(triple_copy)
#             print(new_ft)
            pruned_triples_f += new_ft
        pruned_triples[f-1].append(pruned_triples_f)
        print(pruned_triples[f-1])
        
for f in range(1, 5):
    res[f'pruned_comet{f}'] = pruned_triples[f-1]

1it [00:00, 392.61it/s]

row 0
row 0, sentence 1
row 0, sentence 1, relation xWant
[["Dan 's parents", 'were', 'overweight'], 'xWant', ['to eat less', 'to lose weight', 'to eat healthy', 'to eat healthier', 'to eat more']]
row 0, sentence 1, relation isBefore
[["Dan 's parents", 'were', 'overweight'], 'isBefore', ["Dan 's parents go on a diet", "Dan 's parents go to the gym", 'PersonX eats a lot of junk food', 'PersonX eats a lot of food', "Dan 's parents lose weight"]]
row 0, sentence 1, relation xEffect
[["Dan 's parents", 'were', 'overweight'], 'xEffect', []]
[[[["Dan 's parents", 'were', 'overweight'], 'xWant', [' to buy new clothes', ' to eat less', ' to lose weight', ' to eat healthy', ' to eat less food', ' to eat healthier', ' to eat healthy food', ' to eat less.', ' to lose weight.', ' to eat more']], [["Dan 's parents", 'were', 'overweight'], 'isBefore', [' PersonX eats a lot of junk food.', " Dan's parents go to the gym every day", " Dan 's parents go on a diet", " Dan 's parents go to the gym every

ValueError: Length of values (2) does not match length of index (1)

In [27]:
res

storyid  \
0  9a51198e-96f1-42c3-b09d-a3e1e067d803   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          sentence_1  \
0  [[[Dan 's parents, were, overweight], xWant, [ to buy new clothes,  to eat less,  to lose weight,  to eat healthy,  to eat less food,  to eat healthier,  to eat healthy food,  to eat less.,  to lose weight.,  to eat more]], [[Dan 's parents, were, overweight], isBefore, [ PersonX eats a lot of junk food.,  Dan's parents go to the gym every day,  Dan 's parents go on a diet,  Dan 's parents go to the gym everyday,  Dan 's parents go to the gym,  PersonX eats a lot of food.,  PersonX eats a lot of junk food,  Dan's parents go to the gym,  PersonX eats a lot of food,  Dan 's parents lose weight]], [[Dan 's parents, were, overweight], xEffect, [ has to buy new clothes,  is told to lose weight,  is told to eat less,  Dan's parents are overweight,  Dan's parents are obese,  is told to eat more,  Dan's parents were overweight,  has to eat less,  has to lose weight,  gets teased by parents]]]   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        